In [ ]:
import sklearn.model_selection
import spacy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import scipy.sparse as sp
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
df = pd.read_csv('/content/comp1804_coursework_dataset_23-24 (1).csv')

pd.plotting.register_matplotlib_converters()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.semi_supervised import LabelSpreading
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# PRE-PROCESSING

In [ ]:
# The following text was generated using OpenAI's GPT-3.5 model on March , 2024:
# "The evaluation of artificial intelligence usage from ethical and academic perspectives is crucial to ensure accurate information flow. (Source: https://openai.com/gpt-3.5)"

import string

def remove_punctuation(text):
    """
    Noktalama işaretlerini kaldırmak için bir fonksiyon.
    """
    punctuations = string.punctuation

    no_punctuation_text = ''.join(char for char in text if char not in punctuations)

    return no_punctuation_text

df['paragraph'] = df['paragraph'].apply(remove_punctuation)

In [ ]:
df.isna().sum()

par_id                   0
paragraph                0
has_entity               0
lexicon_count            0
difficult_words         18
last_editor_gender       0
category                61
text_clarity          9338
dtype: int64

In [ ]:
df.drop(columns=['category', 'has_entity','last_editor_gender'], inplace=True)


In [ ]:
df['par_id'].fillna('nan', inplace=True)
df['paragraph'].fillna('nan', inplace=True)
df['lexicon_count'].fillna('nan', inplace=True)
df['difficult_words'].fillna('0', inplace=True)
df['text_clarity'].fillna('to be decided', inplace=True)

In [ ]:
df.isna().sum()

par_id             0
paragraph          0
lexicon_count      0
difficult_words    0
text_clarity       0
dtype: int64

In [ ]:
df_first_100 = df.iloc[9:101]

In [ ]:
df_first_100

,par_id,paragraph,lexicon_count,difficult_words,text_clarity
9,79225005859,Pepys diary provides a firsthand account of th...,139,31.0,to be decided
10,959908012329,Looking ahead the current progress in datatote...,70,22.0,to be decided
11,123020006718,Realtime programs such as simulations flight n...,84,38.0,to be decided
12,113961006578,Darwin was the inventor of several devices tho...,35,8.0,to be decided
13,8471004175,He was among the first British troops into bo...,214,46.0,to be decided
...,...,...,...,...,...
96,4492000140,The original Greek sense a loosening up or re...,44,15.0,to be decided
97,3219003418,beginning with the act of creation by God The ...,16,4.0,to be decided
98,217553007742,The Scratch Lab displays experiments from the ...,31,11.0,to be decided
99,8471004193,With the war over Lloyd George called a genera...,158,37.0,to be decided


In [ ]:
df_first_9 = df.head(9)

In [ ]:
df_first_9

,par_id,paragraph,lexicon_count,difficult_words,text_clarity
0,428209002237,Ramsay was born in Glasgow on 2 October 1852 H...,49,12.0,clear_enough
1,564218010072,It has been widely estimated for at least the ...,166,47.0,not_clear_enough
2,291401001672,He went on to win the Royal Medal of the Royal...,69,18.0,clear_enough
3,31548004883,The changes have altered many underlying assum...,76,27.0,clear_enough
4,50634005146,After these novels were published Disraeli dec...,200,47.0,not_clear_enough
5,220520007895,Philo also proposes that the order in nature m...,137,36.0,not_clear_enough
6,6360003753,This led to the AI spring of the early 2020s ...,116,47.0,clear_enough
7,848119011941,A second form of functionalism is based on the...,42,14.0,not_clear_enough
8,49279000639,The opposite of moral is immoral meaning the w...,51,8.0,clear_enough


# TOKENIZATION

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
from nltk.tokenize import word_tokenize

tokenized_texts = [word_tokenize(text) for text in df_first_9['paragraph']]

for i, tokens in enumerate(tokenized_texts):
    print(f"Metin {i+1} tokenleri: {tokens}")

Metin 1 tokenleri: ['Ramsay', 'was', 'born', 'in', 'Glasgow', 'on', '2', 'October', '1852', 'He', 'was', 'a', 'nephew', 'of', 'the', 'geologist', 'Sir', 'Andrew', 'Ramsay', 'His', 'father', 'William', 'Sr', 'was', 'a', 'civil', 'engineer', 'His', 'mother', 'was', 'Catherine', 'Robertson', 'He', 'studied', 'at', 'Glasgow', 'Academy', 'at', 'the', 'University', 'of', 'Glasgow', 'and', 'at', 'University', 'of', 'Tubingen', 'in', 'Germany']
Metin 2 tokenleri: ['It', 'has', 'been', 'widely', 'estimated', 'for', 'at', 'least', 'the', 'last', 'generation', 'that', 'all', 'the', 'programs', 'and', 'resources', 'devoted', 'to', 'ensuring', 'access', 'to', 'justice', 'address', 'only', '20', 'of', 'the', 'civil', 'legal', 'needs', 'of', 'lowincome', 'people', 'in', 'the', 'United', 'States', 'Drawing', 'on', 'this', 'experience', 'in', 'late', '2011', 'the', 'US', 'governmentfunded', 'Legal', 'Services', 'Corporation', 'decided', 'to', 'convene', 'a', 'summit', 'of', 'leaders', 'to', 'explore', 

In [ ]:
for i, paragraph in enumerate(df_first_9['paragraph']):
    doc = nlp(paragraph)

    print(f"Lemmatized version of paragraph {i+1}:")
    for token in doc:
        print(f"Original token: '{token.text}', Lemma: '{token.lemma_}'")
    print("")
    if i >= 9:
        break

Lemmatized version of paragraph 1:
Original token: 'Ramsay', Lemma: 'Ramsay'
Original token: 'was', Lemma: 'be'
Original token: 'born', Lemma: 'bear'
Original token: 'in', Lemma: 'in'
Original token: 'Glasgow', Lemma: 'Glasgow'
Original token: 'on', Lemma: 'on'
Original token: '2', Lemma: '2'
Original token: 'October', Lemma: 'October'
Original token: '1852', Lemma: '1852'
Original token: 'He', Lemma: 'he'
Original token: 'was', Lemma: 'be'
Original token: 'a', Lemma: 'a'
Original token: 'nephew', Lemma: 'nephew'
Original token: 'of', Lemma: 'of'
Original token: 'the', Lemma: 'the'
Original token: 'geologist', Lemma: 'geologist'
Original token: 'Sir', Lemma: 'Sir'
Original token: 'Andrew', Lemma: 'Andrew'
Original token: 'Ramsay', Lemma: 'Ramsay'
Original token: 'His', Lemma: 'his'
Original token: 'father', Lemma: 'father'
Original token: 'William', Lemma: 'William'
Original token: 'Sr', Lemma: 'Sr'
Original token: 'was', Lemma: 'be'
Original token: 'a', Lemma: 'a'
Original token: 'civ

# Vectorization and Normalization

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

texts = df_first_9['paragraph']

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectors = tfidf_vectorizer.fit_transform(texts)



# count_vectorizer = CountVectorizer()
# count_vectors = count_vectorizer.fit_transform(texts)

print("TF-IDF Vectors shape:", tfidf_vectors.shape) #(9,472) 9 veri sayısı 472 kelime dağarcıgının boyutu yani frekans
# print("Count Vectors shape:", count_vectors.shape)

TF-IDF Vectors shape: (9, 472)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

lexicon_data = df_first_9['lexicon_count']
diffucylt_data = df_first_9['difficult_words']

scaler_lexicon = MinMaxScaler()
scaler_difficult = MinMaxScaler()

lexicon_normalized = scaler_lexicon.fit_transform(lexicon_data.values.reshape(-1, 1))


difficult_words_normalized = scaler_difficult.fit_transform(diffucylt_data.values.reshape(-1, 1))

df_first_9['lexicon_normalized'] = lexicon_normalized
df_first_9['difficult_words_normalized'] = difficult_words_normalized

print("Normalized lexicon data:\n", df_first_9['lexicon_normalized'])
print("\nNormalized difficult data:\n", df_first_9['difficult_words_normalized']) #minmax bütün değerleri 1 ile 0 arasında bir değere ceviriyor

Normalized lexicon data:
 0    0.044304
1    0.784810
2    0.170886
3    0.215190
4    1.000000
5    0.601266
6    0.468354
7    0.000000
8    0.056962
Name: lexicon_normalized, dtype: float64

Normalized difficult data:
 0    0.102564
1    1.000000
2    0.256410
3    0.487179
4    1.000000
5    0.717949
6    1.000000
7    0.153846
8    0.000000
Name: difficult_words_normalized, dtype: float64


<ipython-input-107-cbdd1b038d9a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_first_9['lexicon_normalized'] = lexicon_normalized
<ipython-input-107-cbdd1b038d9a>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_first_9['difficult_words_normalized'] = difficult_words_normalized


In [ ]:
from sklearn.preprocessing import MinMaxScaler

lexicon_data = df_first_100['lexicon_count']
diffucylt_data = df_first_100['difficult_words']

scaler_lexicon = MinMaxScaler()
scaler_difficult = MinMaxScaler()

lexicon_normalized = scaler_lexicon.fit_transform(lexicon_data.values.reshape(-1, 1))


difficult_words_normalized = scaler_difficult.fit_transform(diffucylt_data.values.reshape(-1, 1))

df_first_100['lexicon_normalized'] = lexicon_normalized
df_first_100['difficult_words_normalized'] = difficult_words_normalized

print("Normalized lexicon data:\n", df_first_100['lexicon_normalized'])
print("\nNormalized difficult data:\n", df_first_100['difficult_words_normalized']) #minmax bütün değerleri 1 ile 0 arasında bir değere ceviriyor

Normalized lexicon data:
 9      0.492063
10     0.218254
11     0.273810
12     0.079365
13     0.789683
         ...   
96     0.115079
97     0.003968
98     0.063492
99     0.567460
100    0.210317
Name: lexicon_normalized, Length: 92, dtype: float64

Normalized difficult data:
 9      0.516667
10     0.366667
11     0.633333
12     0.133333
13     0.766667
         ...   
96     0.250000
97     0.066667
98     0.183333
99     0.616667
100    0.283333
Name: difficult_words_normalized, Length: 92, dtype: float64


<ipython-input-108-044bf7652e7e>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_first_100['lexicon_normalized'] = lexicon_normalized
<ipython-input-108-044bf7652e7e>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_first_100['difficult_words_normalized'] = difficult_words_normalized


In [ ]:
df_first_100

,par_id,paragraph,lexicon_count,difficult_words,text_clarity,lexicon_normalized,difficult_words_normalized
9,79225005859,Pepys diary provides a firsthand account of th...,139,31.0,to be decided,0.492063,0.516667
10,959908012329,Looking ahead the current progress in datatote...,70,22.0,to be decided,0.218254,0.366667
11,123020006718,Realtime programs such as simulations flight n...,84,38.0,to be decided,0.273810,0.633333
12,113961006578,Darwin was the inventor of several devices tho...,35,8.0,to be decided,0.079365,0.133333
13,8471004175,He was among the first British troops into bo...,214,46.0,to be decided,0.789683,0.766667
...,...,...,...,...,...,...,...
96,4492000140,The original Greek sense a loosening up or re...,44,15.0,to be decided,0.115079,0.250000
97,3219003418,beginning with the act of creation by God The ...,16,4.0,to be decided,0.003968,0.066667
98,217553007742,The Scratch Lab displays experiments from the ...,31,11.0,to be decided,0.063492,0.183333
99,8471004193,With the war over Lloyd George called a genera...,158,37.0,to be decided,0.567460,0.616667


# Data Splitting

In [ ]:
# df_first_9.drop(columns = ['lexicon_count','difficult_words'], inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

X = df_first_9[['paragraph','lexicon_normalized','difficult_words_normalized']]
y = df_first_9[['text_clarity']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Eğitim kümesi boyutu:", X_train.shape)
print("Test kümesi boyutu:", X_test.shape)

Eğitim kümesi boyutu: (7, 3)
Test kümesi boyutu: (2, 3)


In [ ]:
df_first_9

,par_id,paragraph,lexicon_count,difficult_words,text_clarity,lexicon_normalized,difficult_words_normalized
0,428209002237,Ramsay was born in Glasgow on 2 October 1852 H...,49,12.0,clear_enough,0.044304,0.102564
1,564218010072,It has been widely estimated for at least the ...,166,47.0,not_clear_enough,0.784810,1.000000
2,291401001672,He went on to win the Royal Medal of the Royal...,69,18.0,clear_enough,0.170886,0.256410
3,31548004883,The changes have altered many underlying assum...,76,27.0,clear_enough,0.215190,0.487179
4,50634005146,After these novels were published Disraeli dec...,200,47.0,not_clear_enough,1.000000,1.000000
5,220520007895,Philo also proposes that the order in nature m...,137,36.0,not_clear_enough,0.601266,0.717949
6,6360003753,This led to the AI spring of the early 2020s ...,116,47.0,clear_enough,0.468354,1.000000
7,848119011941,A second form of functionalism is based on the...,42,14.0,not_clear_enough,0.000000,0.153846
8,49279000639,The opposite of moral is immoral meaning the w...,51,8.0,clear_enough,0.056962,0.000000


# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

decision_tree_model = DecisionTreeClassifier()
decision_tree_model.fit(tfidf_vectors, df_first_9['text_clarity'])

train_accuracy = decision_tree_model.score(tfidf_vectors, df_first_9['text_clarity'])
print("Eğitim kümesi doğruluğu:", train_accuracy)

Eğitim kümesi doğruluğu: 1.0


# Model Training for 9 to 100

In [ ]:
texts_100 = df_first_100['paragraph']
tfidf_vectors_100 = tfidf_vectorizer.transform(texts_100)

predictions_100 = decision_tree_model.predict(tfidf_vectors_100)

df_first_100['predicted_target'] = predictions_100

print(df_first_100.head())

          par_id                                          paragraph  \
9    79225005859  Pepys diary provides a firsthand account of th...   
10  959908012329  Looking ahead the current progress in datatote...   
11  123020006718  Realtime programs such as simulations flight n...   
12  113961006578  Darwin was the inventor of several devices tho...   
13    8471004175   He was among the first British troops into bo...   

    lexicon_count difficult_words   text_clarity  lexicon_normalized  \
9             139            31.0  to be decided            0.492063   
10             70            22.0  to be decided            0.218254   
11             84            38.0  to be decided            0.273810   
12             35             8.0  to be decided            0.079365   
13            214            46.0  to be decided            0.789683   

    difficult_words_normalized  predicted_target  
9                     0.516667  not_clear_enough  
10                    0.366667      cl

<ipython-input-114-81f2cd69795a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_first_100['predicted_target'] = predictions_100


In [ ]:
df_first_100

,par_id,paragraph,lexicon_count,difficult_words,text_clarity,lexicon_normalized,difficult_words_normalized,predicted_target
9,79225005859,Pepys diary provides a firsthand account of th...,139,31.0,to be decided,0.492063,0.516667,not_clear_enough
10,959908012329,Looking ahead the current progress in datatote...,70,22.0,to be decided,0.218254,0.366667,clear_enough
11,123020006718,Realtime programs such as simulations flight n...,84,38.0,to be decided,0.273810,0.633333,clear_enough
12,113961006578,Darwin was the inventor of several devices tho...,35,8.0,to be decided,0.079365,0.133333,clear_enough
13,8471004175,He was among the first British troops into bo...,214,46.0,to be decided,0.789683,0.766667,clear_enough
...,...,...,...,...,...,...,...,...
96,4492000140,The original Greek sense a loosening up or re...,44,15.0,to be decided,0.115079,0.250000,clear_enough
97,3219003418,beginning with the act of creation by God The ...,16,4.0,to be decided,0.003968,0.066667,clear_enough
98,217553007742,The Scratch Lab displays experiments from the ...,31,11.0,to be decided,0.063492,0.183333,clear_enough
99,8471004193,With the war over Lloyd George called a genera...,158,37.0,to be decided,0.567460,0.616667,clear_enough


In [ ]:
# The following text was generated using OpenAI's GPT-3.5 model on March , 2024:
# "The evaluation of artificial intelligence usage from ethical and academic perspectives is crucial to ensure accurate information flow. (Source: https://openai.com/gpt-3.5)"

df_combined = pd.concat([df_first_9.reset_index(drop=True), df_first_100.reset_index(drop=True)], axis=0)

print(df_combined.head())


         par_id                                          paragraph  \
0  428209002237  Ramsay was born in Glasgow on 2 October 1852 H...   
1  564218010072  It has been widely estimated for at least the ...   
2  291401001672  He went on to win the Royal Medal of the Royal...   
3   31548004883  The changes have altered many underlying assum...   
4   50634005146  After these novels were published Disraeli dec...   

   lexicon_count difficult_words      text_clarity  lexicon_normalized  \
0             49            12.0      clear_enough            0.044304   
1            166            47.0  not_clear_enough            0.784810   
2             69            18.0      clear_enough            0.170886   
3             76            27.0      clear_enough            0.215190   
4            200            47.0  not_clear_enough            1.000000   

   difficult_words_normalized predicted_target  
0                    0.102564              NaN  
1                    1.000000       

In [ ]:
df_combined

,par_id,paragraph,lexicon_count,difficult_words,text_clarity,lexicon_normalized,difficult_words_normalized,predicted_target
0,428209002237,Ramsay was born in Glasgow on 2 October 1852 H...,49,12.0,clear_enough,0.044304,0.102564,NaN
1,564218010072,It has been widely estimated for at least the ...,166,47.0,not_clear_enough,0.784810,1.000000,NaN
2,291401001672,He went on to win the Royal Medal of the Royal...,69,18.0,clear_enough,0.170886,0.256410,NaN
3,31548004883,The changes have altered many underlying assum...,76,27.0,clear_enough,0.215190,0.487179,NaN
4,50634005146,After these novels were published Disraeli dec...,200,47.0,not_clear_enough,1.000000,1.000000,NaN
...,...,...,...,...,...,...,...,...
87,4492000140,The original Greek sense a loosening up or re...,44,15.0,to be decided,0.115079,0.250000,clear_enough
88,3219003418,beginning with the act of creation by God The ...,16,4.0,to be decided,0.003968,0.066667,clear_enough
89,217553007742,The Scratch Lab displays experiments from the ...,31,11.0,to be decided,0.063492,0.183333,clear_enough
90,8471004193,With the war over Lloyd George called a genera...,158,37.0,to be decided,0.567460,0.616667,clear_enough


In [ ]:
df_combined['text_clarity'].iloc[9:] = df_first_100['predicted_target'].values

print(df_combined.head())

         par_id                                          paragraph  \
0  428209002237  Ramsay was born in Glasgow on 2 October 1852 H...   
1  564218010072  It has been widely estimated for at least the ...   
2  291401001672  He went on to win the Royal Medal of the Royal...   
3   31548004883  The changes have altered many underlying assum...   
4   50634005146  After these novels were published Disraeli dec...   

   lexicon_count difficult_words      text_clarity  lexicon_normalized  \
0             49            12.0      clear_enough            0.044304   
1            166            47.0  not_clear_enough            0.784810   
2             69            18.0      clear_enough            0.170886   
3             76            27.0      clear_enough            0.215190   
4            200            47.0  not_clear_enough            1.000000   

   difficult_words_normalized predicted_target  
0                    0.102564              NaN  
1                    1.000000       

<ipython-input-118-f3dd995eccae>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined['text_clarity'].iloc[9:] = df_first_100['predicted_target'].values


In [ ]:
df_combined

,par_id,paragraph,lexicon_count,difficult_words,text_clarity,lexicon_normalized,difficult_words_normalized,predicted_target
0,428209002237,Ramsay was born in Glasgow on 2 October 1852 H...,49,12.0,clear_enough,0.044304,0.102564,NaN
1,564218010072,It has been widely estimated for at least the ...,166,47.0,not_clear_enough,0.784810,1.000000,NaN
2,291401001672,He went on to win the Royal Medal of the Royal...,69,18.0,clear_enough,0.170886,0.256410,NaN
3,31548004883,The changes have altered many underlying assum...,76,27.0,clear_enough,0.215190,0.487179,NaN
4,50634005146,After these novels were published Disraeli dec...,200,47.0,not_clear_enough,1.000000,1.000000,NaN
...,...,...,...,...,...,...,...,...
87,4492000140,The original Greek sense a loosening up or re...,44,15.0,clear_enough,0.115079,0.250000,clear_enough
88,3219003418,beginning with the act of creation by God The ...,16,4.0,clear_enough,0.003968,0.066667,clear_enough
89,217553007742,The Scratch Lab displays experiments from the ...,31,11.0,clear_enough,0.063492,0.183333,clear_enough
90,8471004193,With the war over Lloyd George called a genera...,158,37.0,clear_enough,0.567460,0.616667,clear_enough


In [ ]:
 df_combined.drop(columns = ['predicted_target'], inplace=True)

In [ ]:
df_combined

,par_id,paragraph,lexicon_count,difficult_words,text_clarity,lexicon_normalized,difficult_words_normalized
0,428209002237,Ramsay was born in Glasgow on 2 October 1852 H...,49,12.0,clear_enough,0.044304,0.102564
1,564218010072,It has been widely estimated for at least the ...,166,47.0,not_clear_enough,0.784810,1.000000
2,291401001672,He went on to win the Royal Medal of the Royal...,69,18.0,clear_enough,0.170886,0.256410
3,31548004883,The changes have altered many underlying assum...,76,27.0,clear_enough,0.215190,0.487179
4,50634005146,After these novels were published Disraeli dec...,200,47.0,not_clear_enough,1.000000,1.000000
...,...,...,...,...,...,...,...
87,4492000140,The original Greek sense a loosening up or re...,44,15.0,clear_enough,0.115079,0.250000
88,3219003418,beginning with the act of creation by God The ...,16,4.0,clear_enough,0.003968,0.066667
89,217553007742,The Scratch Lab displays experiments from the ...,31,11.0,clear_enough,0.063492,0.183333
90,8471004193,With the war over Lloyd George called a genera...,158,37.0,clear_enough,0.567460,0.616667


# Model training for 101 to 500

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

texts = df_combined['paragraph']

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectors = tfidf_vectorizer.fit_transform(texts)



# count_vectorizer = CountVectorizer()
# count_vectors = count_vectorizer.fit_transform(texts)

print("TF-IDF Vectors shape:", tfidf_vectors.shape) #(9,472) 9 veri sayısı 472 kelime dağarcıgının boyutu yani frekans
# print("Count Vectors shape:", count_vectors.shape)

TF-IDF Vectors shape: (101, 2755)


In [ ]:
from sklearn.model_selection import train_test_split

X = df_combined[['paragraph','lexicon_normalized','difficult_words_normalized']]
y = df_combined[['text_clarity']]

# Verileri eğitim ve test kümelerine bölme
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Bölünmüş veri kümelerini kontrol etme
print("Eğitim kümesi boyutu:", X_train.shape)
print("Test kümesi boyutu:", X_test.shape)

Eğitim kümesi boyutu: (80, 3)
Test kümesi boyutu: (21, 3)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

decision_tree_model = DecisionTreeClassifier()
decision_tree_model.fit(tfidf_vectors, df_combined['text_clarity'])

train_accuracy = decision_tree_model.score(tfidf_vectors, df_combined['text_clarity'])
print("Eğitim kümesi doğruluğu:", train_accuracy)

Eğitim kümesi doğruluğu: 1.0


In [ ]:
df_next_500 = df.iloc[101:501]

In [ ]:
df_next_500

,par_id,paragraph,lexicon_count,difficult_words,text_clarity
101,3219003433,These arguments often center on what it means ...,56,17.0,to be decided
102,694781102764,Prof Henry Norris Russell ForMemRS HFRSE FRAS ...,47,15.0,to be decided
103,323165008762,Binney was part of a close Manchester social c...,37,7.0,to be decided
104,7628004101,With its rich set of operators the C language ...,49,12.0,to be decided
105,220520007855,Here he appears to give his support to the ar...,190,51.0,to be decided
...,...,...,...,...,...
496,803253011573,Besides being a scientist John Tyndall was a s...,118,30.0,to be decided
497,77886005794,Liberty is the state of being free within soci...,34,12.0,to be decided
498,835516011856,Swift also has a trailing closure syntax which...,41,11.0,to be decided
499,44742005022,Schmidhuber noted that the resurgence of neura...,107,38.0,to be decided


In [ ]:
from sklearn.preprocessing import MinMaxScaler

lexicon_data = df_next_500['lexicon_count']
diffucylt_data = df_next_500['difficult_words']

scaler_lexicon = MinMaxScaler()
scaler_difficult = MinMaxScaler()

lexicon_normalized = scaler_lexicon.fit_transform(lexicon_data.values.reshape(-1, 1))


difficult_words_normalized = scaler_difficult.fit_transform(diffucylt_data.values.reshape(-1, 1))

df_next_500['lexicon_normalized'] = lexicon_normalized
df_next_500['difficult_words_normalized'] = difficult_words_normalized

print("Normalized lexicon data:\n", df_next_500['lexicon_normalized'])
print("\nNormalized difficult data:\n", df_next_500['difficult_words_normalized']) #minmax bütün değerleri 1 ile 0 arasında bir değere ceviriyor

Normalized lexicon data:
 101    0.134293
102    0.112710
103    0.088729
104    0.117506
105    0.455635
         ...   
496    0.282974
497    0.081535
498    0.098321
499    0.256595
500    0.038369
Name: lexicon_normalized, Length: 400, dtype: float64

Normalized difficult data:
 101    0.204819
102    0.180723
103    0.084337
104    0.144578
105    0.614458
         ...   
496    0.361446
497    0.144578
498    0.132530
499    0.457831
500    0.060241
Name: difficult_words_normalized, Length: 400, dtype: float64


<ipython-input-127-ae53b69c3bda>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_next_500['lexicon_normalized'] = lexicon_normalized
<ipython-input-127-ae53b69c3bda>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_next_500['difficult_words_normalized'] = difficult_words_normalized


In [ ]:
df_next_500

,par_id,paragraph,lexicon_count,difficult_words,text_clarity,lexicon_normalized,difficult_words_normalized
101,3219003433,These arguments often center on what it means ...,56,17.0,to be decided,0.134293,0.204819
102,694781102764,Prof Henry Norris Russell ForMemRS HFRSE FRAS ...,47,15.0,to be decided,0.112710,0.180723
103,323165008762,Binney was part of a close Manchester social c...,37,7.0,to be decided,0.088729,0.084337
104,7628004101,With its rich set of operators the C language ...,49,12.0,to be decided,0.117506,0.144578
105,220520007855,Here he appears to give his support to the ar...,190,51.0,to be decided,0.455635,0.614458
...,...,...,...,...,...,...,...
496,803253011573,Besides being a scientist John Tyndall was a s...,118,30.0,to be decided,0.282974,0.361446
497,77886005794,Liberty is the state of being free within soci...,34,12.0,to be decided,0.081535,0.144578
498,835516011856,Swift also has a trailing closure syntax which...,41,11.0,to be decided,0.098321,0.132530
499,44742005022,Schmidhuber noted that the resurgence of neura...,107,38.0,to be decided,0.256595,0.457831


In [ ]:
texts_500 = df_next_500['paragraph']
tfidf_vectors_500 = tfidf_vectorizer.transform(texts_500)

predictions_500 = decision_tree_model.predict(tfidf_vectors_500)

df_next_500['predicted_target'] = predictions_500

print(df_next_500.head())

           par_id                                          paragraph  \
101    3219003433  These arguments often center on what it means ...   
102  694781102764  Prof Henry Norris Russell ForMemRS HFRSE FRAS ...   
103  323165008762  Binney was part of a close Manchester social c...   
104    7628004101  With its rich set of operators the C language ...   
105  220520007855   Here he appears to give his support to the ar...   

     lexicon_count difficult_words   text_clarity  lexicon_normalized  \
101             56            17.0  to be decided            0.134293   
102             47            15.0  to be decided            0.112710   
103             37             7.0  to be decided            0.088729   
104             49            12.0  to be decided            0.117506   
105            190            51.0  to be decided            0.455635   

     difficult_words_normalized  predicted_target  
101                    0.204819      clear_enough  
102                    0

<ipython-input-129-78c588f8cd8b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_next_500['predicted_target'] = predictions_500


In [ ]:
df_next_500

,par_id,paragraph,lexicon_count,difficult_words,text_clarity,lexicon_normalized,difficult_words_normalized,predicted_target
101,3219003433,These arguments often center on what it means ...,56,17.0,to be decided,0.134293,0.204819,clear_enough
102,694781102764,Prof Henry Norris Russell ForMemRS HFRSE FRAS ...,47,15.0,to be decided,0.112710,0.180723,not_clear_enough
103,323165008762,Binney was part of a close Manchester social c...,37,7.0,to be decided,0.088729,0.084337,clear_enough
104,7628004101,With its rich set of operators the C language ...,49,12.0,to be decided,0.117506,0.144578,clear_enough
105,220520007855,Here he appears to give his support to the ar...,190,51.0,to be decided,0.455635,0.614458,not_clear_enough
...,...,...,...,...,...,...,...,...
496,803253011573,Besides being a scientist John Tyndall was a s...,118,30.0,to be decided,0.282974,0.361446,clear_enough
497,77886005794,Liberty is the state of being free within soci...,34,12.0,to be decided,0.081535,0.144578,clear_enough
498,835516011856,Swift also has a trailing closure syntax which...,41,11.0,to be decided,0.098321,0.132530,clear_enough
499,44742005022,Schmidhuber noted that the resurgence of neura...,107,38.0,to be decided,0.256595,0.457831,clear_enough


In [ ]:
# The following text was generated using OpenAI's GPT-3.5 model on March , 2024:
# "The evaluation of artificial intelligence usage from ethical and academic perspectives is crucial to ensure accurate information flow. (Source: https://openai.com/gpt-3.5)"

df_combined_500 = pd.concat([df_combined.reset_index(drop=True), df_next_500.reset_index(drop=True)], axis=0)

print(df_combined.head())

         par_id                                          paragraph  \
0  428209002237  Ramsay was born in Glasgow on 2 October 1852 H...   
1  564218010072  It has been widely estimated for at least the ...   
2  291401001672  He went on to win the Royal Medal of the Royal...   
3   31548004883  The changes have altered many underlying assum...   
4   50634005146  After these novels were published Disraeli dec...   

   lexicon_count difficult_words      text_clarity  lexicon_normalized  \
0             49            12.0      clear_enough            0.044304   
1            166            47.0  not_clear_enough            0.784810   
2             69            18.0      clear_enough            0.170886   
3             76            27.0      clear_enough            0.215190   
4            200            47.0  not_clear_enough            1.000000   

   difficult_words_normalized  
0                    0.102564  
1                    1.000000  
2                    0.256410  
3     

In [ ]:

df_combined_500['text_clarity'].iloc[101:] = df_next_500['predicted_target'].values

print(df_combined.head())

         par_id                                          paragraph  \
0  428209002237  Ramsay was born in Glasgow on 2 October 1852 H...   
1  564218010072  It has been widely estimated for at least the ...   
2  291401001672  He went on to win the Royal Medal of the Royal...   
3   31548004883  The changes have altered many underlying assum...   
4   50634005146  After these novels were published Disraeli dec...   

   lexicon_count difficult_words      text_clarity  lexicon_normalized  \
0             49            12.0      clear_enough            0.044304   
1            166            47.0  not_clear_enough            0.784810   
2             69            18.0      clear_enough            0.170886   
3             76            27.0      clear_enough            0.215190   
4            200            47.0  not_clear_enough            1.000000   

   difficult_words_normalized  
0                    0.102564  
1                    1.000000  
2                    0.256410  
3     

<ipython-input-133-3959c9c4303f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined_500['text_clarity'].iloc[101:] = df_next_500['predicted_target'].values


In [ ]:
df_combined_500

,par_id,paragraph,lexicon_count,difficult_words,text_clarity,lexicon_normalized,difficult_words_normalized,predicted_target
0,428209002237,Ramsay was born in Glasgow on 2 October 1852 H...,49,12.0,clear_enough,0.044304,0.102564,NaN
1,564218010072,It has been widely estimated for at least the ...,166,47.0,not_clear_enough,0.784810,1.000000,NaN
2,291401001672,He went on to win the Royal Medal of the Royal...,69,18.0,clear_enough,0.170886,0.256410,NaN
3,31548004883,The changes have altered many underlying assum...,76,27.0,clear_enough,0.215190,0.487179,NaN
4,50634005146,After these novels were published Disraeli dec...,200,47.0,not_clear_enough,1.000000,1.000000,NaN
...,...,...,...,...,...,...,...,...
395,803253011573,Besides being a scientist John Tyndall was a s...,118,30.0,clear_enough,0.282974,0.361446,clear_enough
396,77886005794,Liberty is the state of being free within soci...,34,12.0,clear_enough,0.081535,0.144578,clear_enough
397,835516011856,Swift also has a trailing closure syntax which...,41,11.0,clear_enough,0.098321,0.132530,clear_enough
398,44742005022,Schmidhuber noted that the resurgence of neura...,107,38.0,clear_enough,0.256595,0.457831,clear_enough


In [ ]:
 df_combined_500.drop(columns = ['predicted_target'], inplace=True)

In [ ]:
df_combined_500

,par_id,paragraph,lexicon_count,difficult_words,text_clarity,lexicon_normalized,difficult_words_normalized
0,428209002237,Ramsay was born in Glasgow on 2 October 1852 H...,49,12.0,clear_enough,0.044304,0.102564
1,564218010072,It has been widely estimated for at least the ...,166,47.0,not_clear_enough,0.784810,1.000000
2,291401001672,He went on to win the Royal Medal of the Royal...,69,18.0,clear_enough,0.170886,0.256410
3,31548004883,The changes have altered many underlying assum...,76,27.0,clear_enough,0.215190,0.487179
4,50634005146,After these novels were published Disraeli dec...,200,47.0,not_clear_enough,1.000000,1.000000
...,...,...,...,...,...,...,...
395,803253011573,Besides being a scientist John Tyndall was a s...,118,30.0,clear_enough,0.282974,0.361446
396,77886005794,Liberty is the state of being free within soci...,34,12.0,clear_enough,0.081535,0.144578
397,835516011856,Swift also has a trailing closure syntax which...,41,11.0,clear_enough,0.098321,0.132530
398,44742005022,Schmidhuber noted that the resurgence of neura...,107,38.0,clear_enough,0.256595,0.457831


# Model Training for 501 to 1000

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

texts = df_combined_500['paragraph']

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectors = tfidf_vectorizer.fit_transform(texts)



# count_vectorizer = CountVectorizer()
# count_vectors = count_vectorizer.fit_transform(texts)

print("TF-IDF Vectors shape:", tfidf_vectors.shape) #(9,472) 9 veri sayısı 472 kelime dağarcıgının boyutu yani frekans
# print("Count Vectors shape:", count_vectors.shape)

TF-IDF Vectors shape: (501, 8351)


In [ ]:
from sklearn.model_selection import train_test_split

X = df_combined_500[['paragraph','lexicon_normalized','difficult_words_normalized']]
y = df_combined_500[['text_clarity']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Eğitim kümesi boyutu:", X_train.shape)
print("Test kümesi boyutu:", X_test.shape)

Eğitim kümesi boyutu: (400, 3)
Test kümesi boyutu: (101, 3)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

decision_tree_model = DecisionTreeClassifier()
decision_tree_model.fit(tfidf_vectors, df_combined_500['text_clarity'])

train_accuracy = decision_tree_model.score(tfidf_vectors, df_combined_500['text_clarity'])
print("Eğitim kümesi doğruluğu:", train_accuracy)

Eğitim kümesi doğruluğu: 1.0


In [ ]:
df_next_1000 = df.iloc[501:1001]

In [ ]:
df_next_1000

,par_id,paragraph,lexicon_count,difficult_words,text_clarity
501,1004853003152,Wang Yangming disagreed with the teachings of ...,33,12.0,to be decided
502,62612000750,Another advantage of median is that it can be ...,57,17.0,to be decided
503,8471004170,Churchill was born on 30 November 1874 at his ...,134,29.0,to be decided
504,109181000911,On 11 February 1800 Herschel was testing filte...,31,7.0,to be decided
505,52793005327,Wellesley had by now acquired the experience o...,75,23.0,to be decided
...,...,...,...,...,...
996,5123003692,Newton was a fellow of Trinity College and the...,153,38.0,to be decided
997,1032461012862,Contentbased recommender systems can also incl...,117,40.0,to be decided
998,85003277,The main philosophical discipline studying exi...,113,20.0,to be decided
999,835516011864,To access the value inside assuming it is not ...,25,7.0,to be decided


In [ ]:
from sklearn.preprocessing import MinMaxScaler

lexicon_data = df_next_1000['lexicon_count']
diffucylt_data = df_next_1000['difficult_words']

scaler_lexicon = MinMaxScaler()
scaler_difficult = MinMaxScaler()

lexicon_normalized = scaler_lexicon.fit_transform(lexicon_data.values.reshape(-1, 1))


difficult_words_normalized = scaler_difficult.fit_transform(diffucylt_data.values.reshape(-1, 1))

df_next_1000['lexicon_normalized'] = lexicon_normalized
df_next_1000['difficult_words_normalized'] = difficult_words_normalized

print("Normalized lexicon data:\n", df_next_1000['lexicon_normalized'])
print("\nNormalized difficult data:\n", df_next_1000['difficult_words_normalized']) #minmax bütün değerleri 1 ile 0 arasında bir değere ceviriyor

Normalized lexicon data:
 501     0.0825
502     0.1425
503     0.3350
504     0.0775
505     0.1875
         ...  
996     0.3825
997     0.2925
998     0.2825
999     0.0625
1000    0.3550
Name: lexicon_normalized, Length: 500, dtype: float64

Normalized difficult data:
 501     0.107143
502     0.151786
503     0.258929
504     0.062500
505     0.205357
          ...   
996     0.339286
997     0.357143
998     0.178571
999     0.062500
1000    0.383929
Name: difficult_words_normalized, Length: 500, dtype: float64


<ipython-input-142-64cfc33479c6>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_next_1000['lexicon_normalized'] = lexicon_normalized
<ipython-input-142-64cfc33479c6>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_next_1000['difficult_words_normalized'] = difficult_words_normalized


In [ ]:
df_next_1000

,par_id,paragraph,lexicon_count,difficult_words,text_clarity,lexicon_normalized,difficult_words_normalized
501,1004853003152,Wang Yangming disagreed with the teachings of ...,33,12.0,to be decided,0.0825,0.107143
502,62612000750,Another advantage of median is that it can be ...,57,17.0,to be decided,0.1425,0.151786
503,8471004170,Churchill was born on 30 November 1874 at his ...,134,29.0,to be decided,0.3350,0.258929
504,109181000911,On 11 February 1800 Herschel was testing filte...,31,7.0,to be decided,0.0775,0.062500
505,52793005327,Wellesley had by now acquired the experience o...,75,23.0,to be decided,0.1875,0.205357
...,...,...,...,...,...,...,...
996,5123003692,Newton was a fellow of Trinity College and the...,153,38.0,to be decided,0.3825,0.339286
997,1032461012862,Contentbased recommender systems can also incl...,117,40.0,to be decided,0.2925,0.357143
998,85003277,The main philosophical discipline studying exi...,113,20.0,to be decided,0.2825,0.178571
999,835516011864,To access the value inside assuming it is not ...,25,7.0,to be decided,0.0625,0.062500


In [ ]:
texts_1000 = df_next_1000['paragraph']
tfidf_vectors_1000 = tfidf_vectorizer.transform(texts_1000)

predictions_1000 = decision_tree_model.predict(tfidf_vectors_1000)

df_next_1000['predicted_target'] = predictions_1000

print(df_next_1000.head())

            par_id                                          paragraph  \
501  1004853003152  Wang Yangming disagreed with the teachings of ...   
502    62612000750  Another advantage of median is that it can be ...   
503     8471004170  Churchill was born on 30 November 1874 at his ...   
504   109181000911  On 11 February 1800 Herschel was testing filte...   
505    52793005327  Wellesley had by now acquired the experience o...   

     lexicon_count difficult_words   text_clarity  lexicon_normalized  \
501             33            12.0  to be decided              0.0825   
502             57            17.0  to be decided              0.1425   
503            134            29.0  to be decided              0.3350   
504             31             7.0  to be decided              0.0775   
505             75            23.0  to be decided              0.1875   

     difficult_words_normalized predicted_target  
501                    0.107143     clear_enough  
502                 

<ipython-input-144-182a38e4ed3a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_next_1000['predicted_target'] = predictions_1000


In [ ]:
# The following text was generated using OpenAI's GPT-3.5 model on March , 2024:
# "The evaluation of artificial intelligence usage from ethical and academic perspectives is crucial to ensure accurate information flow. (Source: https://openai.com/gpt-3.5)"

df_combined_1000 = pd.concat([df_combined_500.reset_index(drop=True), df_next_1000.reset_index(drop=True)], axis=0)

print(df_combined.head())

         par_id                                          paragraph  \
0  428209002237  Ramsay was born in Glasgow on 2 October 1852 H...   
1  564218010072  It has been widely estimated for at least the ...   
2  291401001672  He went on to win the Royal Medal of the Royal...   
3   31548004883  The changes have altered many underlying assum...   
4   50634005146  After these novels were published Disraeli dec...   

   lexicon_count difficult_words      text_clarity  lexicon_normalized  \
0             49            12.0      clear_enough            0.044304   
1            166            47.0  not_clear_enough            0.784810   
2             69            18.0      clear_enough            0.170886   
3             76            27.0      clear_enough            0.215190   
4            200            47.0  not_clear_enough            1.000000   

   difficult_words_normalized  
0                    0.102564  
1                    1.000000  
2                    0.256410  
3     

In [ ]:
df_combined_1000['text_clarity'].iloc[501:] = df_next_1000['predicted_target'].values

print(df_combined.head())

         par_id                                          paragraph  \
0  428209002237  Ramsay was born in Glasgow on 2 October 1852 H...   
1  564218010072  It has been widely estimated for at least the ...   
2  291401001672  He went on to win the Royal Medal of the Royal...   
3   31548004883  The changes have altered many underlying assum...   
4   50634005146  After these novels were published Disraeli dec...   

   lexicon_count difficult_words      text_clarity  lexicon_normalized  \
0             49            12.0      clear_enough            0.044304   
1            166            47.0  not_clear_enough            0.784810   
2             69            18.0      clear_enough            0.170886   
3             76            27.0      clear_enough            0.215190   
4            200            47.0  not_clear_enough            1.000000   

   difficult_words_normalized  
0                    0.102564  
1                    1.000000  
2                    0.256410  
3     

<ipython-input-146-bf3e3d0a083e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined_1000['text_clarity'].iloc[501:] = df_next_1000['predicted_target'].values


In [ ]:
df_combined_1000.drop(columns = ['predicted_target'], inplace=True)

In [ ]:
df_combined_1000

,par_id,paragraph,lexicon_count,difficult_words,text_clarity,lexicon_normalized,difficult_words_normalized
0,428209002237,Ramsay was born in Glasgow on 2 October 1852 H...,49,12.0,clear_enough,0.044304,0.102564
1,564218010072,It has been widely estimated for at least the ...,166,47.0,not_clear_enough,0.784810,1.000000
2,291401001672,He went on to win the Royal Medal of the Royal...,69,18.0,clear_enough,0.170886,0.256410
3,31548004883,The changes have altered many underlying assum...,76,27.0,clear_enough,0.215190,0.487179
4,50634005146,After these novels were published Disraeli dec...,200,47.0,not_clear_enough,1.000000,1.000000
...,...,...,...,...,...,...,...
495,5123003692,Newton was a fellow of Trinity College and the...,153,38.0,clear_enough,0.382500,0.339286
496,1032461012862,Contentbased recommender systems can also incl...,117,40.0,clear_enough,0.292500,0.357143
497,85003277,The main philosophical discipline studying exi...,113,20.0,clear_enough,0.282500,0.178571
498,835516011864,To access the value inside assuming it is not ...,25,7.0,clear_enough,0.062500,0.062500


# Model Training for 1001 to 3000

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

texts = df_combined_1000['paragraph']

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectors = tfidf_vectorizer.fit_transform(texts)



# count_vectorizer = CountVectorizer()
# count_vectors = count_vectorizer.fit_transform(texts)

print("TF-IDF Vectors shape:", tfidf_vectors.shape) #(9,472) 9 veri sayısı 472 kelime dağarcıgının boyutu yani frekans
# print("Count Vectors shape:", count_vectors.shape)

TF-IDF Vectors shape: (1001, 12556)


In [ ]:
from sklearn.model_selection import train_test_split

X = df_combined_1000[['paragraph','lexicon_normalized','difficult_words_normalized']]
y = df_combined_1000[['text_clarity']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Eğitim kümesi boyutu:", X_train.shape)
print("Test kümesi boyutu:", X_test.shape)

Eğitim kümesi boyutu: (800, 3)
Test kümesi boyutu: (201, 3)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

decision_tree_model = DecisionTreeClassifier()
decision_tree_model.fit(tfidf_vectors, df_combined_1000['text_clarity'])

train_accuracy = decision_tree_model.score(tfidf_vectors, df_combined_1000['text_clarity'])
print("Eğitim kümesi doğruluğu:", train_accuracy)

Eğitim kümesi doğruluğu: 1.0


In [ ]:
df_next_3000 = df.iloc[1001:3001]

In [ ]:
df_next_3000

,par_id,paragraph,lexicon_count,difficult_words,text_clarity
1001,985985012515,Can a juvenile suspect in Pakistan who is 13 y...,16,5.0,to be decided
1002,263343008146,Banks left Oxford for Chelsea in December 1763...,163,34.0,to be decided
1003,366875002059,Paul Berg was an American biochemist He was ...,33,10.0,to be decided
1004,6360003840,about 800000 AIrelated US job openings existed...,75,32.0,to be decided
1005,268438008213,This is useful guidance but does not of course...,26,4.0,to be decided
...,...,...,...,...,...
2996,273398008268,Some philosophers argue that some kinds of sp...,52,13.0,to be decided
2997,141864106972,1,0,0.0,to be decided
2998,161437007090,Olivebacked euphonia Two species of reptiles a...,60,16.0,to be decided
2999,110509106385,A suite of tools for developing applications w...,19,7.0,to be decided


In [ ]:
from sklearn.preprocessing import MinMaxScaler

lexicon_data = df_next_3000['lexicon_count']
diffucylt_data = df_next_3000['difficult_words']

scaler_lexicon = MinMaxScaler()
scaler_difficult = MinMaxScaler()

lexicon_normalized = scaler_lexicon.fit_transform(lexicon_data.values.reshape(-1, 1))


difficult_words_normalized = scaler_difficult.fit_transform(diffucylt_data.values.reshape(-1, 1))

df_next_3000['lexicon_normalized'] = lexicon_normalized
df_next_3000['difficult_words_normalized'] = difficult_words_normalized

print("Normalized lexicon data:\n", df_next_3000['lexicon_normalized'])
print("\nNormalized difficult data:\n", df_next_3000['difficult_words_normalized']) #minmax bütün değerleri 1 ile 0 arasında bir değere ceviriyor

Normalized lexicon data:
 1001    0.028520
1002    0.290553
1003    0.058824
1004    0.133690
1005    0.046346
          ...   
2996    0.092692
2997    0.000000
2998    0.106952
2999    0.033868
3000    0.067736
Name: lexicon_normalized, Length: 2000, dtype: float64

Normalized difficult data:
 1001    0.035971
1002    0.244604
1003    0.071942
1004    0.230216
1005    0.028777
          ...   
2996    0.093525
2997    0.000000
2998    0.115108
2999    0.050360
3000    0.079137
Name: difficult_words_normalized, Length: 2000, dtype: float64


<ipython-input-154-40a444dfef63>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_next_3000['lexicon_normalized'] = lexicon_normalized
<ipython-input-154-40a444dfef63>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_next_3000['difficult_words_normalized'] = difficult_words_normalized


In [ ]:
df_next_3000

,par_id,paragraph,lexicon_count,difficult_words,text_clarity,lexicon_normalized,difficult_words_normalized
1001,985985012515,Can a juvenile suspect in Pakistan who is 13 y...,16,5.0,to be decided,0.028520,0.035971
1002,263343008146,Banks left Oxford for Chelsea in December 1763...,163,34.0,to be decided,0.290553,0.244604
1003,366875002059,Paul Berg was an American biochemist He was ...,33,10.0,to be decided,0.058824,0.071942
1004,6360003840,about 800000 AIrelated US job openings existed...,75,32.0,to be decided,0.133690,0.230216
1005,268438008213,This is useful guidance but does not of course...,26,4.0,to be decided,0.046346,0.028777
...,...,...,...,...,...,...,...
2996,273398008268,Some philosophers argue that some kinds of sp...,52,13.0,to be decided,0.092692,0.093525
2997,141864106972,1,0,0.0,to be decided,0.000000,0.000000
2998,161437007090,Olivebacked euphonia Two species of reptiles a...,60,16.0,to be decided,0.106952,0.115108
2999,110509106385,A suite of tools for developing applications w...,19,7.0,to be decided,0.033868,0.050360


In [ ]:
texts_3000 = df_next_3000['paragraph']
tfidf_vectors_3000 = tfidf_vectorizer.transform(texts_3000)

predictions_3000 = decision_tree_model.predict(tfidf_vectors_3000)

df_next_3000['predicted_target'] = predictions_3000

print(df_next_3000.head())

            par_id                                          paragraph  \
1001  985985012515  Can a juvenile suspect in Pakistan who is 13 y...   
1002  263343008146  Banks left Oxford for Chelsea in December 1763...   
1003  366875002059  Paul Berg  was an American biochemist  He was ...   
1004    6360003840  about 800000 AIrelated US job openings existed...   
1005  268438008213  This is useful guidance but does not of course...   

      lexicon_count difficult_words   text_clarity  lexicon_normalized  \
1001             16             5.0  to be decided            0.028520   
1002            163            34.0  to be decided            0.290553   
1003             33            10.0  to be decided            0.058824   
1004             75            32.0  to be decided            0.133690   
1005             26             4.0  to be decided            0.046346   

      difficult_words_normalized  predicted_target  
1001                    0.035971      clear_enough  
1002      

<ipython-input-156-4c3012e58c87>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_next_3000['predicted_target'] = predictions_3000


In [ ]:
df_next_3000

,par_id,paragraph,lexicon_count,difficult_words,text_clarity,lexicon_normalized,difficult_words_normalized,predicted_target
1001,985985012515,Can a juvenile suspect in Pakistan who is 13 y...,16,5.0,to be decided,0.028520,0.035971,clear_enough
1002,263343008146,Banks left Oxford for Chelsea in December 1763...,163,34.0,to be decided,0.290553,0.244604,not_clear_enough
1003,366875002059,Paul Berg was an American biochemist He was ...,33,10.0,to be decided,0.058824,0.071942,not_clear_enough
1004,6360003840,about 800000 AIrelated US job openings existed...,75,32.0,to be decided,0.133690,0.230216,clear_enough
1005,268438008213,This is useful guidance but does not of course...,26,4.0,to be decided,0.046346,0.028777,clear_enough
...,...,...,...,...,...,...,...,...
2996,273398008268,Some philosophers argue that some kinds of sp...,52,13.0,to be decided,0.092692,0.093525,clear_enough
2997,141864106972,1,0,0.0,to be decided,0.000000,0.000000,clear_enough
2998,161437007090,Olivebacked euphonia Two species of reptiles a...,60,16.0,to be decided,0.106952,0.115108,clear_enough
2999,110509106385,A suite of tools for developing applications w...,19,7.0,to be decided,0.033868,0.050360,clear_enough


In [ ]:
# The following text was generated using OpenAI's GPT-3.5 model on March , 2024:
# "The evaluation of artificial intelligence usage from ethical and academic perspectives is crucial to ensure accurate information flow. (Source: https://openai.com/gpt-3.5)"

df_combined_3000 = pd.concat([df_combined_1000.reset_index(drop=True), df_next_3000.reset_index(drop=True)], axis=0)

print(df_combined_3000.head())

         par_id                                          paragraph  \
0  428209002237  Ramsay was born in Glasgow on 2 October 1852 H...   
1  564218010072  It has been widely estimated for at least the ...   
2  291401001672  He went on to win the Royal Medal of the Royal...   
3   31548004883  The changes have altered many underlying assum...   
4   50634005146  After these novels were published Disraeli dec...   

   lexicon_count difficult_words      text_clarity  lexicon_normalized  \
0             49            12.0      clear_enough            0.044304   
1            166            47.0  not_clear_enough            0.784810   
2             69            18.0      clear_enough            0.170886   
3             76            27.0      clear_enough            0.215190   
4            200            47.0  not_clear_enough            1.000000   

   difficult_words_normalized predicted_target  
0                    0.102564              NaN  
1                    1.000000       

In [ ]:
df_combined_3000['text_clarity'].iloc[1001:] = df_next_3000['predicted_target'].values

print(df_combined.head())

         par_id                                          paragraph  \
0  428209002237  Ramsay was born in Glasgow on 2 October 1852 H...   
1  564218010072  It has been widely estimated for at least the ...   
2  291401001672  He went on to win the Royal Medal of the Royal...   
3   31548004883  The changes have altered many underlying assum...   
4   50634005146  After these novels were published Disraeli dec...   

   lexicon_count difficult_words      text_clarity  lexicon_normalized  \
0             49            12.0      clear_enough            0.044304   
1            166            47.0  not_clear_enough            0.784810   
2             69            18.0      clear_enough            0.170886   
3             76            27.0      clear_enough            0.215190   
4            200            47.0  not_clear_enough            1.000000   

   difficult_words_normalized  
0                    0.102564  
1                    1.000000  
2                    0.256410  
3     

<ipython-input-159-b238211d00a4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined_3000['text_clarity'].iloc[1001:] = df_next_3000['predicted_target'].values


In [ ]:
df_combined_3000.drop(columns = ['predicted_target'], inplace=True)

In [ ]:
df_combined_3000

,par_id,paragraph,lexicon_count,difficult_words,text_clarity,lexicon_normalized,difficult_words_normalized
0,428209002237,Ramsay was born in Glasgow on 2 October 1852 H...,49,12.0,clear_enough,0.044304,0.102564
1,564218010072,It has been widely estimated for at least the ...,166,47.0,not_clear_enough,0.784810,1.000000
2,291401001672,He went on to win the Royal Medal of the Royal...,69,18.0,clear_enough,0.170886,0.256410
3,31548004883,The changes have altered many underlying assum...,76,27.0,clear_enough,0.215190,0.487179
4,50634005146,After these novels were published Disraeli dec...,200,47.0,not_clear_enough,1.000000,1.000000
...,...,...,...,...,...,...,...
1995,273398008268,Some philosophers argue that some kinds of sp...,52,13.0,clear_enough,0.092692,0.093525
1996,141864106972,1,0,0.0,clear_enough,0.000000,0.000000
1997,161437007090,Olivebacked euphonia Two species of reptiles a...,60,16.0,clear_enough,0.106952,0.115108
1998,110509106385,A suite of tools for developing applications w...,19,7.0,clear_enough,0.033868,0.050360


# Model Training for 3001 to 9346

In [ ]:
df

,par_id,paragraph,lexicon_count,difficult_words,text_clarity
0,428209002237,Ramsay was born in Glasgow on 2 October 1852 H...,49,12.0,clear_enough
1,564218010072,It has been widely estimated for at least the ...,166,47.0,not_clear_enough
2,291401001672,He went on to win the Royal Medal of the Royal...,69,18.0,clear_enough
3,31548004883,The changes have altered many underlying assum...,76,27.0,clear_enough
4,50634005146,After these novels were published Disraeli dec...,200,47.0,not_clear_enough
...,...,...,...,...,...
9342,200523007578,Sir Joseph Dalton Hooker was a British botani...,54,16.0,to be decided
9343,50634005203,As successful invasions of India generally cam...,217,46.0,to be decided
9344,6437003939,Hawking was born on 8 January 1942 in Oxford t...,91,27.0,to be decided
9345,725259002815,More complex tasks can be making custom data e...,47,15.0,to be decided


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

texts = df_combined_3000['paragraph']

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectors = tfidf_vectorizer.fit_transform(texts)



# count_vectorizer = CountVectorizer()
# count_vectors = count_vectorizer.fit_transform(texts)

print("TF-IDF Vectors shape:", tfidf_vectors.shape) #(9,472) 9 veri sayısı 472 kelime dağarcıgının boyutu yani frekans
# print("Count Vectors shape:", count_vectors.shape)

TF-IDF Vectors shape: (3001, 23154)


In [ ]:
from sklearn.model_selection import train_test_split

X = df_combined_3000[['paragraph','lexicon_normalized','difficult_words_normalized']]
y = df_combined_3000[['text_clarity']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Eğitim kümesi boyutu:", X_train.shape)
print("Test kümesi boyutu:", X_test.shape)

Eğitim kümesi boyutu: (2400, 3)
Test kümesi boyutu: (601, 3)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

decision_tree_model = DecisionTreeClassifier()
decision_tree_model.fit(tfidf_vectors, df_combined_3000['text_clarity'])

train_accuracy = decision_tree_model.score(tfidf_vectors, df_combined_3000['text_clarity'])
print("Eğitim kümesi doğruluğu:", train_accuracy)

Eğitim kümesi doğruluğu: 1.0


In [ ]:
df_final = df.iloc[3001:]

In [ ]:
df_final

,par_id,paragraph,lexicon_count,difficult_words,text_clarity
3001,133020006741,Most of what is known about sophists comes fro...,82,15.0,to be decided
3002,663523010959,Image reconstruction is the reconstruction of ...,39,17.0,to be decided
3003,794896011493,King married Harriet Lethbridge in 1817 prior ...,28,7.0,to be decided
3004,657766010769,For my own part I may say that of all the gent...,169,37.0,to be decided
3005,123020006662,nooperation instructions in a later pass or th...,36,15.0,to be decided
...,...,...,...,...,...
9342,200523007578,Sir Joseph Dalton Hooker was a British botani...,54,16.0,to be decided
9343,50634005203,As successful invasions of India generally cam...,217,46.0,to be decided
9344,6437003939,Hawking was born on 8 January 1942 in Oxford t...,91,27.0,to be decided
9345,725259002815,More complex tasks can be making custom data e...,47,15.0,to be decided


In [ ]:
from sklearn.preprocessing import MinMaxScaler

lexicon_data = df_final['lexicon_count']
diffucylt_data = df_final['difficult_words']

scaler_lexicon = MinMaxScaler()
scaler_difficult = MinMaxScaler()

lexicon_normalized = scaler_lexicon.fit_transform(lexicon_data.values.reshape(-1, 1))


difficult_words_normalized = scaler_difficult.fit_transform(diffucylt_data.values.reshape(-1, 1))

df_final['lexicon_normalized'] = lexicon_normalized
df_final['difficult_words_normalized'] = difficult_words_normalized

print("Normalized lexicon data:\n", df_final['lexicon_normalized'])
print("\nNormalized difficult data:\n", df_final['difficult_words_normalized']) #minmax bütün değerleri 1 ile 0 arasında bir değere ceviriyor

Normalized lexicon data:
 3001    0.125574
3002    0.059724
3003    0.042879
3004    0.258806
3005    0.055130
          ...   
9342    0.082695
9343    0.332312
9344    0.139357
9345    0.071975
9346    0.133231
Name: lexicon_normalized, Length: 6346, dtype: float64

Normalized difficult data:
 3001    0.104895
3002    0.118881
3003    0.048951
3004    0.258741
3005    0.104895
          ...   
9342    0.111888
9343    0.321678
9344    0.188811
9345    0.104895
9346    0.055944
Name: difficult_words_normalized, Length: 6346, dtype: float64


<ipython-input-168-6d3877589792>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['lexicon_normalized'] = lexicon_normalized
<ipython-input-168-6d3877589792>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['difficult_words_normalized'] = difficult_words_normalized


In [ ]:
texts_final = df_final['paragraph']
tfidf_vectors_final = tfidf_vectorizer.transform(texts_final)

predictions_final = decision_tree_model.predict(tfidf_vectors_final)

df_final['predicted_target'] = predictions_final

print(df_final.head())

            par_id                                          paragraph  \
3001  133020006741  Most of what is known about sophists comes fro...   
3002  663523010959  Image reconstruction is the reconstruction of ...   
3003  794896011493  King married Harriet Lethbridge in 1817 prior ...   
3004  657766010769  For my own part I may say that of all the gent...   
3005  123020006662  nooperation instructions in a later pass or th...   

      lexicon_count difficult_words   text_clarity  lexicon_normalized  \
3001             82            15.0  to be decided            0.125574   
3002             39            17.0  to be decided            0.059724   
3003             28             7.0  to be decided            0.042879   
3004            169            37.0  to be decided            0.258806   
3005             36            15.0  to be decided            0.055130   

      difficult_words_normalized  predicted_target  
3001                    0.104895  not_clear_enough  
3002      

<ipython-input-184-a3e44546aa1d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['predicted_target'] = predictions_final


In [ ]:
df_final

,par_id,paragraph,lexicon_count,difficult_words,text_clarity,lexicon_normalized,difficult_words_normalized,predicted_target
3001,133020006741,Most of what is known about sophists comes fro...,82,15.0,to be decided,0.125574,0.104895,not_clear_enough
3002,663523010959,Image reconstruction is the reconstruction of ...,39,17.0,to be decided,0.059724,0.118881,clear_enough
3003,794896011493,King married Harriet Lethbridge in 1817 prior ...,28,7.0,to be decided,0.042879,0.048951,clear_enough
3004,657766010769,For my own part I may say that of all the gent...,169,37.0,to be decided,0.258806,0.258741,not_clear_enough
3005,123020006662,nooperation instructions in a later pass or th...,36,15.0,to be decided,0.055130,0.104895,not_clear_enough
...,...,...,...,...,...,...,...,...
9342,200523007578,Sir Joseph Dalton Hooker was a British botani...,54,16.0,to be decided,0.082695,0.111888,clear_enough
9343,50634005203,As successful invasions of India generally cam...,217,46.0,to be decided,0.332312,0.321678,clear_enough
9344,6437003939,Hawking was born on 8 January 1942 in Oxford t...,91,27.0,to be decided,0.139357,0.188811,clear_enough
9345,725259002815,More complex tasks can be making custom data e...,47,15.0,to be decided,0.071975,0.104895,not_clear_enough


In [ ]:
# The following text was generated using OpenAI's GPT-3.5 model on March , 2024:
# "The evaluation of artificial intelligence usage from ethical and academic perspectives is crucial to ensure accurate information flow. (Source: https://openai.com/gpt-3.5)"

df_combined_final = pd.concat([df_combined_3000.reset_index(drop=True), df_final.reset_index(drop=True)], axis=0)

print(df_combined.head())

         par_id                                          paragraph  \
0  428209002237  Ramsay was born in Glasgow on 2 October 1852 H...   
1  564218010072  It has been widely estimated for at least the ...   
2  291401001672  He went on to win the Royal Medal of the Royal...   
3   31548004883  The changes have altered many underlying assum...   
4   50634005146  After these novels were published Disraeli dec...   

   lexicon_count difficult_words      text_clarity  lexicon_normalized  \
0             49            12.0      clear_enough            0.044304   
1            166            47.0  not_clear_enough            0.784810   
2             69            18.0      clear_enough            0.170886   
3             76            27.0      clear_enough            0.215190   
4            200            47.0  not_clear_enough            1.000000   

   difficult_words_normalized  
0                    0.102564  
1                    1.000000  
2                    0.256410  
3     

In [ ]:
df_combined_final['text_clarity'].iloc[3001:] = df_final['predicted_target'].values

print(df_combined.head())

         par_id                                          paragraph  \
0  428209002237  Ramsay was born in Glasgow on 2 October 1852 H...   
1  564218010072  It has been widely estimated for at least the ...   
2  291401001672  He went on to win the Royal Medal of the Royal...   
3   31548004883  The changes have altered many underlying assum...   
4   50634005146  After these novels were published Disraeli dec...   

   lexicon_count difficult_words      text_clarity  lexicon_normalized  \
0             49            12.0      clear_enough            0.044304   
1            166            47.0  not_clear_enough            0.784810   
2             69            18.0      clear_enough            0.170886   
3             76            27.0      clear_enough            0.215190   
4            200            47.0  not_clear_enough            1.000000   

   difficult_words_normalized  
0                    0.102564  
1                    1.000000  
2                    0.256410  
3     

<ipython-input-172-d6316ccdce91>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined_final['text_clarity'].iloc[3001:] = df_final['predicted_target'].values


In [ ]:
df_combined_final.drop(columns = ['predicted_target'], inplace=True)

In [ ]:
df_combined_final

,par_id,paragraph,lexicon_count,difficult_words,text_clarity,lexicon_normalized,difficult_words_normalized
0,428209002237,Ramsay was born in Glasgow on 2 October 1852 H...,49,12.0,clear_enough,0.044304,0.102564
1,564218010072,It has been widely estimated for at least the ...,166,47.0,not_clear_enough,0.784810,1.000000
2,291401001672,He went on to win the Royal Medal of the Royal...,69,18.0,clear_enough,0.170886,0.256410
3,31548004883,The changes have altered many underlying assum...,76,27.0,clear_enough,0.215190,0.487179
4,50634005146,After these novels were published Disraeli dec...,200,47.0,not_clear_enough,1.000000,1.000000
...,...,...,...,...,...,...,...
6341,200523007578,Sir Joseph Dalton Hooker was a British botani...,54,16.0,clear_enough,0.082695,0.111888
6342,50634005203,As successful invasions of India generally cam...,217,46.0,clear_enough,0.332312,0.321678
6343,6437003939,Hawking was born on 8 January 1942 in Oxford t...,91,27.0,clear_enough,0.139357,0.188811
6344,725259002815,More complex tasks can be making custom data e...,47,15.0,not_clear_enough,0.071975,0.104895


In [ ]:
df_combined_final.isna().sum()

par_id                        0
paragraph                     0
lexicon_count                 0
difficult_words               0
text_clarity                  0
lexicon_normalized            0
difficult_words_normalized    0
dtype: int64

In [ ]:
counts = df_combined_final['text_clarity'].value_counts()
print(counts)

clear_enough        6394
not_clear_enough    2953
Name: text_clarity, dtype: int64


# Evaluation of Final Dataset

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# texts = df_combined_final['paragraph']

# tfidf_vectorizer = TfidfVectorizer()
# tfidf_vectors = tfidf_vectorizer.fit_transform(texts)

# print("TF-IDF Vectors shape:", tfidf_vectors.shape)

TF-IDF Vectors shape: (9347, 41328)


In [ ]:
# from sklearn.model_selection import train_test_split

# X = df_combined_final[['paragraph','lexicon_normalized','difficult_words_normalized']]
# y = df_combined_final[['text_clarity']]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# print("Eğitim kümesi boyutu:", X_train.shape)
# print("Test kümesi boyutu:", X_test.shape)

Eğitim kümesi boyutu: (7477, 3)
Test kümesi boyutu: (1870, 3)


In [ ]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import classification_report

# decision_tree_model = DecisionTreeClassifier()
# decision_tree_model.fit(tfidf_vectors, df_combined_final['text_clarity'])

# train_accuracy = decision_tree_model.score(tfidf_vectors, df_combined_final['text_clarity'])
# print("Eğitim kümesi doğruluğu:", train_accuracy)


Eğitim kümesi doğruluğu: 1.0


In [ ]:
# from sklearn.preprocessing import MinMaxScaler

# lexicon_data = df_combined_final['lexicon_count']
# diffucylt_data = df_combined_final['difficult_words']

# scaler_lexicon = MinMaxScaler()
# scaler_difficult = MinMaxScaler()

# lexicon_normalized = scaler_lexicon.fit_transform(lexicon_data.values.reshape(-1, 1))


# difficult_words_normalized = scaler_difficult.fit_transform(diffucylt_data.values.reshape(-1, 1))

# df_combined_final['lexicon_normalized'] = lexicon_normalized
# df_combined_final['difficult_words_normalized'] = difficult_words_normalized

# print("Normalized lexicon data:\n", df_combined_final['lexicon_normalized'])
# print("\nNormalized difficult data:\n", df_combined_final['difficult_words_normalized']) #minmax bütün değerleri 1 ile 0 arasında bir değere ceviriyor

Normalized lexicon data:
 0       0.075038
1       0.254211
2       0.105666
3       0.116386
4       0.306279
          ...   
6341    0.082695
6342    0.332312
6343    0.139357
6344    0.071975
6345    0.133231
Name: lexicon_normalized, Length: 9347, dtype: float64

Normalized difficult data:
 0       0.083916
1       0.328671
2       0.125874
3       0.188811
4       0.328671
          ...   
6341    0.111888
6342    0.321678
6343    0.188811
6344    0.104895
6345    0.055944
Name: difficult_words_normalized, Length: 9347, dtype: float64


In [ ]:
# texts_final = df_combined_final['paragraph']
# tfidf_vectors_final = tfidf_vectorizer.transform(texts_final)

# predictions_final = decision_tree_model.predict(tfidf_vectors_final)

# df_combined_final['predicted_target'] = predictions_final

# print(df_combined_final.head())

# report = classification_report(df_combined_final['text_clarity'], df_combined_final['predicted_target'])

# print(report)

# print(df_combined_final.head())

         par_id                                          paragraph  \
0  428209002237  Ramsay was born in Glasgow on 2 October 1852 H...   
1  564218010072  It has been widely estimated for at least the ...   
2  291401001672  He went on to win the Royal Medal of the Royal...   
3   31548004883  The changes have altered many underlying assum...   
4   50634005146  After these novels were published Disraeli dec...   

   lexicon_count difficult_words      text_clarity  lexicon_normalized  \
0             49            12.0      clear_enough            0.075038   
1            166            47.0  not_clear_enough            0.254211   
2             69            18.0      clear_enough            0.105666   
3             76            27.0      clear_enough            0.116386   
4            200            47.0  not_clear_enough            0.306279   

   difficult_words_normalized  predicted_target  
0                    0.083916      clear_enough  
1                    0.328671  not